In [ ]:
## ref : https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/optimize-llama-2-gptq.ipynb 

In [ ]:
!sudo pip install -q transformers --upgrade

In [ ]:
!sudo -H pip install auto-gptq --no-cache-dir

In [ ]:
!sudo -H pip install --upgrade optimum

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.gptq import GPTQQuantizer, load_quantized_model
import torch